In [ ]:
from app import scraping
from app import geos
from app import bundling
from app import input
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Individual match scraping
### Flight and Accommodation
The purpose of the individual scraping is to check the latest price of the selected match (different to in bulk scraping, which takes a lot of time, this individual is much faster)

Please exchange the the following filepaths for your own, leading to the text file used for selecting the match

In [ ]:
# Internal Input: Read the internal file and extract data into a data frame for processing
file_path = '/Users/danielavandasova/Library/CloudStorage/OneDrive-Personal/Documents/3_IES/8-Semestr/Data Analysis in Python/Python_Project/internal-data.xlsx'
data_frame = read_excel_to_df(file_path)

In [ ]:
# User input: Read the txt file and extract the primary code
file_path = '/Users/danielavandasova/Library/CloudStorage/OneDrive-Personal/Documents/3_IES/8-Semestr/Data Analysis in Python/Python_Project/input.txt' #path to input file
primary_code = read_code(file_path, data_frame)

In [ ]:
# From data_frame extract the row corresponding to the primary code
df = data_frame.loc[data_frame.iloc[:, 0] == primary_code]
#print(row)

In [ ]:
# Initialize the WebDriver (e.g., Chrome)
driver = webdriver.Chrome()

# Loop through each row in the DataFrame
min_prices = []  # To store the minimum prices for each city and date combination
for index, row in df.iterrows():
    city = row['flight.Code']  # Use the column name 'flight.Code' for the city
    departure_date = row['date'].strftime('%Y-%m-%d')  # Assuming 'date' is in a proper date format

    # Calculate the return date as departure date + 2 days
    return_date = (row['date'] + timedelta(days=2)).strftime('%Y-%m-%d')
    
    # Scrape the minimum price for the city and date combination
    min_price = scrape_min_price(city, departure_date, return_date)
    
    # Append the minimum price to the list
    min_prices.append(min_price)

    print(min_price)

# Add the minimum price as a new column in the original DataFrame
df['Min Price'] = min_prices

# Close the WebDriver
driver.quit()

# Display the updated DataFrame
print(df)

In [ ]:
# Define lists to hold the accommodation data for each row
accommodation_standard = []
accommodation_superior = []
accommodation_luxurious = []

driver = webdriver.Chrome()
driver.maximize_window()

# Loop through each row in the DataFrame to get location and dates
for index, row in df.iterrows():
    # Calculate the difference between the current date and the 'date' in the row
    if row['date'] < (datetime.now() + timedelta(days=14)):
        continue  # Skip this iteration if the date is less than 14 days from today

    location = row['accom.Code']  # Assuming 'accom.Code' contains the location identifier
    departure_date = row['date'].strftime('%Y-%m-%d')  # Assuming 'date' is a date column
    return_date = (row['date'] + timedelta(days=2)).strftime('%Y-%m-%d') 

    city = row['city']  # Assuming 'city' contains the city name
    
    # Scrape accommodation data
    accommodation_data = scrape_accommodation_data(location, departure_date, return_date)
    accommodation_data["Location"] = accommodation_data["Location"] + ", " + city
    
    accommodation_data = accommodation_data[~accommodation_data['Location'].str.contains("Nearby - ", na=False)]
    current_venue = row["venue"] 

    places = get_coordinates(accommodation_data['Location'])

    places_df = pd.DataFrame(places, columns=["Place", "Latitude", "Longitude"])

    # Create a mask for valid coordinates (i.e., filter out "Location not found")
    valid_mask = places_df["Latitude"] != "Location not found"

    accommodation_data = accommodation_data.reset_index(drop=True)

    # Ensure valid_mask has the same index as the original accommodation_data
    valid_mask = valid_mask & accommodation_data.index.isin(places_df.index)

    # Apply the mask to filter both places_df and accommodation_data
    places_filtered = places_df[valid_mask].reset_index(drop=True)
    accommodation_data = accommodation_data[valid_mask].reset_index(drop=True)

    geolocator = Nominatim(user_agent="Geopy Library")
    current_venue = geolocator.geocode(current_venue)
    base_coords = (current_venue.latitude, current_venue.longitude) if current_venue else "Location not found"

    base_latitude = base_coords[0]
    base_longitude = base_coords[1] 


    distances = []
    for i in range(len(places_filtered)):
        # Access latitude and longitude using pandas iloc to reference rows
        latitude = places_filtered.iloc[i]["Latitude"]
        longitude = places_filtered.iloc[i]["Longitude"]
        
        # Calculate distance between base coordinates and other places
        dist = geodesic((base_latitude, base_longitude), (latitude, longitude)).kilometers
        distances.append(dist)

    # Add distances to accommodation_data
    accommodation_data["Distance"] = distances

    # Select the Standard, Superior, and Luxurious bundles
    Standard_bundle, Superior_bundle, Luxurious_bundle = select_accommodation_bundles(accommodation_data)

    # Append the accommodation names and prices to the corresponding lists
    accommodation_standard.append({
        'Name': Standard_bundle['Name'],
        'Price': Standard_bundle['Price'],
        'Rating': Standard_bundle['Rating']
    })
    accommodation_superior.append({
        'Name': Superior_bundle['Name'],
        'Price': Superior_bundle['Price'],
        'Rating': Superior_bundle['Rating']
    })
    accommodation_luxurious.append({
        'Name': Luxurious_bundle['Name'],
        'Price': Luxurious_bundle['Price'],
        'Rating': Luxurious_bundle['Rating']
    })

# Add the accommodation bundles to the original DataFrame
df['Standard Accommodation'] = [x['Name'] for x in accommodation_standard]
df['Superior Accommodation'] = [x['Name'] for x in accommodation_superior]
df['Luxurious Accommodation'] = [x['Name'] for x in accommodation_luxurious]

df['Standard Price'] = [x['Price'] for x in accommodation_standard]
df['Superior Price'] = [x['Price'] for x in accommodation_superior]
df['Luxurious Price'] = [x['Price'] for x in accommodation_luxurious]

df['Standard Rating'] = [x['Rating'] for x in accommodation_standard]
df['Superior Rating'] = [x['Rating'] for x in accommodation_superior]
df['Luxurious Rating'] = [x['Rating'] for x in accommodation_luxurious]

# Save the updated DataFrame with accommodation data

# Close the WebDriver
driver.quit()


In [ ]:
print(df)

## Visuals
### Flight and Accommodation Pricing Trends 
The purpose of this graph is to show how much in advance is it best to book a stay to see a match. The graph contains price averages of each match day in our database.
standard, superior, luxurious

In [ ]:
url = "XXX DATA CO NEMAME"
response = requests.get(url)
df = pd.read_excel(BytesIO(response.content))
df

In [ ]:
# Create a new DataFrame that contains only the 'date', 'Standard Price Total', 'Superior Price Total', and 'Luxurious Price Total' columns
accommodation_data_1 = accommodation_data[['date', 'Standard Price Total', 'Superior Price Total', 'Luxurious Price Total']]
#print(accommodation_data_1)

In [ ]:
# Group the data by 'date' and calculate the mean price for each accommodation type
accommodation_data_1 = accommodation_data_1.groupby('date').mean()
#print(accommodation_data_1)

In [ ]:
# Consider each date as series and accomodation type as category
# Plot a bar chart that has 3 columns for each date (accomodation type) and the height of the columns is the mean price
accommodation_data_1.plot(kind='bar', figsize=(12, 6))
plt.title('Mean Price of Accommodation Types Over Time')
plt.ylabel('Mean Price')
# Instead of dates on the x-axis, show the dates as strings
plt.xticks(ticks=range(len(accommodation_data_1.index)), labels=[date.strftime('%Y-%m-%d') for date in accommodation_data_1.index], rotation=45)
plt.show()

In [ ]:
# Create a new DataFrame that contains only the 'event', 'Standard Price Total', 'Superior Price Total', and 'Luxurious Price Total' columns
accommodation_data_2 = accommodation_data[['event', 'Standard Price Total', 'Superior Price Total', 'Luxurious Price Total']]
#print(accommodation_data_2)

In [ ]:
# Create an empty list, ake the 'event' column and separate it into two columns ('Home' and 'Away') separated by the ' vs. ' string
event_list = []
for event in accommodation_data_2['event']:
    event_list.append(event.split(' vs. '))
    # Appent the 'Standard Price Total', 'Superior Price Total', and 'Luxurious Price Total' columns to the list
    event_list[-1].extend(accommodation_data_2.loc[accommodation_data_2['event'] == event, ['Standard Price Total', 'Superior Price Total', 'Luxurious Price Total']].values[0])
#print(event_list)

# Create a DataFrame from the list
event_df = pd.DataFrame(event_list, columns=['Home', 'Away', 'Standard Price Total', 'Superior Price Total', 'Luxurious Price Total'])
event_df = event_df.groupby(['Home', 'Away']).mean().unstack()
#print(event_df)

In [ ]:
# Create a heatmap that shows Home team on y-axis, Away team on x-axis, and the mean price Standard accommodation as the value
# Plot the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(event_df['Standard Price Total'], annot=False, cmap='coolwarm')
plt.title('Mean Price of Standard Accommodation for Home and Away Teams')
plt.show()

# Create a heatmap that shows Home team on y-axis, Away team on x-axis, and the mean price Superior accommodation as the value
# Plot the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(event_df['Superior Price Total'], annot=False, cmap='coolwarm')
plt.title('Mean Price of Superior Accommodation for Home and Away Teams')
plt.show()

# Create a heatmap that shows Home team on y-axis, Away team on x-axis, and the mean price Luxurious accommodation as the value
# Plot the heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(event_df['Luxurious Price Total'], annot=False, cmap='coolwarm')
plt.title('Mean Price of Luxurious Accommodation for Home and Away Teams')
plt.show()